Miscelaneo
http://www.bom.gov.au/

https://www.portalminero.com/pages/viewpage.action?pageId=6661853

https://datos.bancomundial.org/indicator/NY.GDP.MINR.RT.ZS?locations=AU

# Informe técnico tercera evaluación Mineria de Datos

Integrantes:

*   Felipe Salas
*   Brian Urbina
*   Marcelo Montecino



1. Datos a trabajar


      *   2009-2015 🆗
      *   Se comparan resultados con 2016
      *   No se toman los años 2007, 2008, 2017,2018 🆗

 2. Del caso de negocio asociado al hallazgo, debe considerar 2 variables de target (Categórica y Continua)

 3. Para cada una de las variables seleccionadas**


      *   Imputar los datos necesarios con 3 técnicas distintas y presentar las matrices de correlación de c/u de ellas
      *   Argumentar cual de esas imputaciones es la seleccionada

HASTA AQUI ES EL AVANCE DEL Domingo 9
<HR>
  4. Generar 3 modelos para c/u de las variables y comparar los scoring
  5. Seleccionar el modelo ganador
  6. Se definirá como criterio de éxito de la siguiente forma

      *  Predicción para 7 dias, debe al menos 5 días tener un margen de error de un 8% (mas o menos)
  7. Comparación con los datos del año 2016








JUPYTER PARA ENTREGA 3

WORD CON
INFORME CON ENTREGA 1, 2 Y 3
INTRODUCCION, INDICE, CONCLUSION


## 1.- Comprensión del negocio.
---

1. **Contexto General de Australia**

Australia es un país vasto con una superficie de aproximadamente 7.741.220 km², lo que lo convierte en el sexto país más grande del mundo. La población se concentra mayoritariamente en las zonas urbanas, especialmente en las ciudades de Sídney, Melbourne, Brisbane, Perth y Adelaide. El país está dividido en seis estados (New South Wales, Victoria, Queensland, South Australia, Western Australia y Tasmania) y dos territorios (Northern Territory y Australian Capital Territory).

2. **Clima y Geografía**

Australia es el continente más seco después de la Antártida, con un clima predominantemente desértico y semiárido, aunque hay variaciones climáticas significativas:
- **Norte:** Clima tropical con una estación seca en invierno y una húmeda en verano.
- **Sureste y Sudoeste:** Clima templado con cuatro estaciones.
- **Centro:** Predomina el clima desértico.
- **Gran Cordillera Divisoria:** Extensa cadena montañosa que influye en los patrones de precipitación y alberga los ríos más caudalosos del país, como el Murray y el Darling.

3. **Sector Minero en Australia**

El sector minero es uno de los pilares económicos de Australia, representando alrededor del 10% del PIB con un valor aproximado de 148.000 millones de AUD. Australia es uno de los principales productores y exportadores de minerales y productos energéticos a nivel mundial. Los productos mineros más significativos incluyen Mineral de Hierro, Carbón, Bauxita, Alúmina, Zinc y Gas Natural Licuado (GNL).


4. **Factores Climáticos Relevantes para la Minería**

Los fenómenos climáticos extremos como sequías, inundaciones, ciclones tropicales, vendavales e incendios forestales tienen un impacto significativo en las operaciones mineras. Estos eventos pueden causar interrupciones en la producción, daños a la infraestructura y afectaciones en la logística y transporte.

5. **Objetivo del Análisis**

El objetivo del análisis es utilizar los datos meteorológicos diarios proporcionados para explorar patrones climáticos que puedan predecir eventos que afecten al sector minero, específicamente la variable objetivo **RainTomorrow** (si hay lluvia al día siguiente - No / Sí) y la variable de riesgo **RISK_MM** (cuánta lluvia registrada en milímetros). Este análisis permitirá:
- Identificar patrones y tendencias climáticas que afectan la producción y exportación minera.
- Prever eventos climáticos extremos que puedan interrumpir las operaciones mineras.
- Proporcionar recomendaciones basadas en datos para mejorar la resiliencia del sector minero ante fenómenos climáticos adversos.

6. **Preguntas Clave para el Análisis**

- ¿Cómo han afectado los eventos climáticos extremos (inundaciones, sequías, ciclones) a la producción y exportación minera en los últimos años?
- ¿Cuáles son los patrones estacionales de lluvia y cómo influyen en las operaciones mineras?
- ¿Qué medidas pueden adoptarse para mitigar los impactos negativos de los eventos climáticos en el sector minero?





## 2.- Compresión de los datos.
---





In [0]:
pip install tabulate

In [0]:
#weather_spark_df = spark.read.option("header", "true").csv("abfss://raw@cs2100320032141b0ad.dfs.core.windows.net/weatherAUS.csv")
#df_spark_df = spark.read.option("header", "true").csv("abfss://raw@cs2100320032141b0ad.dfs.core.windows.net/df.csv")
#df_test_spark_df = spark.read.option("header", "true").csv("abfss://raw@cs2100320032141b0ad.dfs.core.windows.net/df_test.csv")
df_resultados = spark.read.option("header", "true").csv("abfss://raw@cs2100320032141b0ad.dfs.core.windows.net/resultados.csv")

"""
display(weather_spark_df)
df = weather_spark_df.toPandas()
display(df)

display(df_spark_df)
df_spark_df.createOrReplaceTempView("df_spark_df")
display(df_test_spark_df)
df_test_spark_df.createOrReplaceTempView("df_test_spark_df")
"""

df_resultados.createOrReplaceTempView("df_resultados")

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- selección de fechas para homologación: se tomó fechas al azar en una lista para calcular el margen de error
-- en vista y consideración de que debemos llegar a calcular el margen, inputamos los nulos con 0.1 para prevenir que la división se indefina
with datos as (
  select *, (error_absoluto / if(RISK_MM_Real = 0, 0.1, RISK_MM_Real)) as algo
  from df_resultados
),
margen_detalle as (
  select fecha, algo*100 as margen_error
  from datos
  where algo between 0.06 and 0.19
  order by margen_error asc
),
margen_promedio as (
  select fecha, algo*100 as margen_error
  from datos
  where algo between 0.06 and 0.09
  order by margen_error asc
)

select sum(margen_error)/count(*)
from margen_promedio



In [0]:
%sql
select Date, cast(RISK_MM as double) as risk_mm_number
from df_spark_df
where cast(RISK_MM as double) < 33.9

Databricks visualization. Run in Databricks to view.

In [0]:
#Importar librerias
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Cargar el conjunto de datos
#df = pd.read_csv('weatherAUS.csv')

In [0]:
# Obtener los tipos de datos de cada columna
dtypes = df.dtypes

# Separar variables categóricas y numéricas
categoricas = df.select_dtypes(include=['object'])
numericas = df.select_dtypes(exclude=['object'])

# Descripción de variables categóricas y numéricas
desc_categoricas = categoricas.describe()
desc_numericas = numericas.describe()

# Agregar una columna para el nombre de las filas en la descripción
desc_categoricas.insert(0, 'Estadístico', desc_categoricas.index)
desc_numericas.insert(0, 'Estadístico', desc_numericas.index)

# Convertir los DataFrames en tablas con tabulate
tabla_categoricas = tabulate(desc_categoricas, headers='keys', tablefmt='pretty', showindex=False)
tabla_numericas = tabulate(desc_numericas, headers='keys', tablefmt='pretty', showindex=False)

# Imprimir tablas
print("Información del Conjunto de Datos:")
print("----------------------------------")
print("Variables Categóricas:")
print(tabla_categoricas)
print("\nVariables Numéricas:")
print(tabla_numericas)
print("\nNúmero total de entradas:", len(df))


In [0]:
# Calcular el total de valores nulos en toda la data
total_nulos = df.isnull().sum().sum()

# Calcular el total de datos en toda la data
total_datos = df.size

# Imprimir los resultados

print("TOTAL DE VALORES NULOS")
for feature in df.columns:
   total_feature_nulos = df[feature].isna().sum()
   porcentaje_nulos = (total_feature_nulos / df.shape[0]) * 100
   print(f'Total de valores nulos de {feature}: {total_feature_nulos} ({porcentaje_nulos:.2f}%)')

print("------------------------------------------------")
print("Total de valores nulos en toda la data:", total_nulos)
print("Total de datos en toda la data:", total_datos)


In [0]:
# Calcular el total de valores nulos en toda la data
total_nulos = df.isnull().sum().sum()

# Calcular el total de datos en toda la data
total_datos = df.size

# Función para identificar outliers utilizando el método del rango intercuartílico (IQR)
def identificar_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

# Imprimir los resultados
print("TOTAL DE OUTLIERS")
for feature in df.columns:
    if df[feature].dtype in ['float64', 'int64']:  # Considerar solo columnas numéricas
        total_feature_outliers = identificar_outliers(df[feature])
        porcentaje_outliers = (total_feature_outliers / df.shape[0]) * 100
        print(f'Total de outliers en {feature}: {total_feature_outliers} ({porcentaje_outliers:.2f}%)')

print("------------------------------------------------")
print("Total de valores nulos en toda la data:", total_nulos)
print("Total de datos en toda la data:", total_datos)


### Matriz de Correlación
---

In [0]:
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm', 'Cloud9am', 'Cloud3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()


### Variables Objetivo
---

#### Variable Objetivo Categórica: ""RainTomorrow""

In [0]:
df["RainTomorrow"].describe()

#### Variable Objetivo Numérica: ""RISK_MM""

In [0]:
df["RISK_MM"].describe()

In [0]:
# Obtener las variables numéricas
numericas = df.select_dtypes(exclude=['object'])

# Calcular la matriz de correlación
corr = numericas.corr()

# Seleccionar las correlaciones más interesantes
corr_seleccionadas = corr[corr['RISK_MM'].abs() > 0.5]

# Crear un mapa de calor de la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_seleccionadas, cmap='coolwarm', annot=True)
plt.title('Mapa de calor de las correlaciones más interesantes')
plt.show()


### **Hallazgo 1: Falta de datos**
---

El gráfico muestra la cantidad de filas de datos por año. Se observa que los años 2007, 2008, 2017 y 2018 presentan una cantidad significativamente menor de datos en comparación con los demás años.

Un conjunto de datos con un número reducido de observaciones puede no ser representativo de la población general, lo que puede llevar a conclusiones erróneas o poco precisas. Por lo mismo se opto por eliminar dichos años para mejorar la precisión del análisis

In [0]:
# Convertir la columna 'Date' a formato de fecha
df['Date'] = pd.to_datetime(df['Date'])

# Obtener el año de cada fila
df['Year'] = df['Date'].dt.year

# Contar la cantidad de filas por año
rows_by_year = df['Year'].value_counts().sort_index()

# Crear un índice de todos los años presentes en los datos
all_years = range(df['Year'].min(), df['Year'].max() + 1)

# Rellenar los años faltantes con ceros
rows_by_year = rows_by_year.reindex(all_years, fill_value=0)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(rows_by_year.index, rows_by_year, color='skyblue')

# Agregar el número de filas encima de cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom')

plt.title('Cantidad de filas por año')
plt.xlabel('Año')
plt.ylabel('Cantidad de filas')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()



### **Hallazgo 2: La niña**
---
**Explicación:**

El fenómeno de la Niña es cuando el agua en el océano Pacífico cerca del ecuador se enfría más de lo normal, lo que puede provocar cambios en el clima, como más lluvias en algunas áreas, sequías en otras y más tormentas tropicales.

Este fenómeno se dio en Australia durante los años 2010 y 2011, el cual afecto mayormente al area de Queensland, provocando una de las peores inundaciones de la ultima decada en este país.

*   **Inundaciones de Queensland:** Queensland experimentó una de las peores inundaciones en su historia debido a lluvias torrenciales. Esta inundación afectó gravemente la minería, especialmente las minas de carbón, ya que muchas quedaron inundadas y hubo interrupciones significativas en la producción, lo que causo perdidas aproximadas de $4 billones .

*   **Ciclón Yasi:** Este ciclón, uno de los más poderosos en la historia de Australia, golpeó Queensland en febrero de 2011. Causó daños considerables en las infraestructuras mineras y en la cadena de suministro.


**Justificación:**
El fenómeno se respalda en nuestros datos debido a que, como se muestra en el gráfico, existe una gran cantidad de lluvia en esos años a diferencia de los otros.



**Fuentes:**

[MDPI](https://www.mdpi.com/2073-4441/3/4/1149)

[Geoscience Australia](https://www.ga.gov.au)

In [0]:
# Convertir la columna 'Date' a tipo datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtrar los datos para incluir solo los años 2009 a 2015
df_filtered = df[(df['Date'].dt.year >= 2009) & (df['Date'].dt.year <= 2015)]

# Sumar la cantidad de lluvia por año
rain_by_year = df_filtered.groupby(df_filtered['Date'].dt.year)['RISK_MM'].sum().reset_index()

# Ordenar los años de mayor a menor cantidad de lluvia (y mantener ese orden)
rain_by_year_sorted = rain_by_year.sort_values(by='RISK_MM', ascending=False)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(range(len(rain_by_year_sorted)), rain_by_year_sorted['RISK_MM'], color='blue')  # Usamos range(len()) para asignar posiciones
plt.xlabel('Año (ordenado por cantidad de lluvia)')
plt.ylabel('Cantidad de lluvia (mm)')
plt.title('Cantidad de lluvia por año (2009-2015)')
plt.xticks(range(len(rain_by_year_sorted)), rain_by_year_sorted['Date'])  # Asignamos etiquetas según el nuevo orden
plt.grid(axis='y')
plt.show()


### **Hallazgo 3: Sequías significas entre el año 2013 y 2015**
---

El gráfico muestra la cantidad promedio anual de lluvia en Australia entre 2009 y 2015. Se observa una tendencia a la baja en la precipitación durante este período, con un mínimo en el año 2014.

De acuerdo a información real, Australia experimentó una sequía severa entre 2009 y 2015. Esta sequía fue la peor en la historia registrada del país y afectó a una gran parte del territorio.

La sequía tuvo un impacto significativo en la industria minera de Australia, de la siguiente manera:

* **Reducción de la producción:** La falta de agua afectó la disponibilidad de agua para los procesos de minería, lo que llevó a una reducción en la producción de minerales.
* **Aumento de los costos:** La sequía también provocó un aumento en los costos de la minería, ya que las empresas tuvieron que invertir más en la búsqueda de agua y en el transporte de agua a los sitios mineros.
* **Impactos ambientales:** La sequía también tuvo un impacto negativo en el medio ambiente, ya que la minería requiere grandes cantidades de agua.


In [0]:

# Convertir la columna 'Date' a tipo datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtrar los datos para incluir solo los años 2009 a 2015
df_filtered = df[(df['Date'].dt.year >= 2009) & (df['Date'].dt.year <= 2015)]

# Calcular la cantidad de lluvia promedio por año
rain_by_year = df_filtered.groupby(df_filtered['Date'].dt.year)['Rainfall'].mean().reset_index()
rain_by_year = df_filtered.groupby(df_filtered['Date'].dt.year)['Rainfall'].mean().reset_index()
# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
plt.plot(rain_by_year['Date'], rain_by_year['Rainfall'], marker='o', color='b', linestyle='-')
plt.xlabel('Año')
plt.ylabel('Rainfall (mm)')
plt.title('Rainfall promedio por año (2009-2015)')
plt.grid(True)
plt.xticks(rain_by_year['Date'])
plt.show()


In [0]:

# Convertir la columna 'Date' a formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filtrar datos para los años de interés
years_group1 = [2014]
years_group2 = [2011]

df_group1 = df[df['Date'].dt.year.isin(years_group1)]
df_group2 = df[df['Date'].dt.year.isin(years_group2)]

nulos_2014 = df_group1.isnull().sum().sum()
nulos_2011 = df_group2 .isnull().sum().sum()

# Definir columnas de interés para el análisis (puedes ajustar según necesites)
columns_of_interest = ['MinTemp', 'MaxTemp', 'Rainfall','RISK_MM',  'Sunshine', 'WindGustSpeed', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']

# Crear matrices de correlación para cada grupo
correlation_matrix_group1 = df_group1[columns_of_interest].corr()
correlation_matrix_group2 = df_group2[columns_of_interest].corr()

# Configurar el tamaño del gráfico
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.heatmap(correlation_matrix_group1, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title(f'Matriz de Correlación 2014\nValores nulos: {nulos_2014}')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Mapa de calor para el grupo 2 (2010, 2011)
plt.subplot(1, 2, 2)
sns.heatmap(correlation_matrix_group2, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title(f'Matriz de Correlación 2011\nValores nulos: {nulos_2011}')
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Ajustar el diseño del subplot
plt.tight_layout()

# Mostrar los mapas de calor
plt.show()

In [0]:

# Supongamos que ya tienes df_filtered, rain_by_year y wind_speed_by_year definidos como en tu código

# Calcular la presión promedio por año
pressure_by_year = df_filtered.groupby(df_filtered['Date'].dt.year)['Pressure9am'].mean().reset_index()
wind_speed_by_year = df_filtered.groupby(df_filtered['Date'].dt.year)['WindGustSpeed'].mean().reset_index()

# Crear una figura con tres subplots (uno encima del otro)
fig, ( ax2, ax3) = plt.subplots(nrows=2, ncols=1, figsize=(10, 12), sharex=True)


# Gráfico para promedio de velocidad del viento por año
ax2.plot(wind_speed_by_year['Date'], wind_speed_by_year['WindGustSpeed'], marker='s', color='g', linestyle='-', label='WindGustSpeed')
ax2.set_ylabel('Promedio de velocidad del viento')
ax2.legend()
ax2.grid(True)
ax2.tick_params(axis='x', rotation=45)

# Gráfico para promedio de presión a las 9 am por año
ax3.plot(pressure_by_year['Date'], pressure_by_year['Pressure9am'], marker='^', color='r', linestyle='-', label='Pressure9am')
ax3.set_xlabel('Año')
ax3.set_ylabel('Promedio de presión (9 am)')
ax3.legend()
ax3.grid(True)
ax3.tick_params(axis='x', rotation=45)

# Ajustar el diseño de los subplots para evitar superposiciones
plt.tight_layout()

# Mostrar los gráficos
plt.show()




## 3.- Preparación de los datos.
---

Filtrar fechas para comprensión

In [0]:
# Convertir la columna 'Date' a formato DateTime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Filtrar el DataFrame para excluir los años 2007, 2008, 2017 y 2018
df = df[(df['Date'].dt.year >= 2009) & (df['Date'].dt.year <= 2015)]  # Filtrar por año


Filtrar por ubicacion

In [0]:
# Definir las ubicaciones para cada región
new_south_wales = ('Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree', 'Newcastle', 'NorahHead', 'Penrith', 'Richmond', 'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown', 'Wollongong', 'NorfolkIsland')

# Crear datasets para cada región
df = df[df['Location'].isin(new_south_wales)]

###Eliminar Nulos que esten por sobre el 37%

In [0]:

# Calcular el total de valores nulos en toda la data
total_nulos = df.isnull().sum().sum()

# Calcular el total de datos en toda la data
total_datos = df.size

# Identificar las columnas que tienen un porcentaje de nulos mayor al 37%
threshold = 36
columns_to_drop = [feature for feature in df.columns if (df[feature].isna().sum() / df.shape[0]) * 100 > threshold]

# Eliminar las columnas identificadas del DataFrame original
df.drop(columns=columns_to_drop, inplace=True)

# Imprimir los resultados
print("TOTAL DE VALORES NULOS")
for feature in df.columns:
    total_feature_nulos = df[feature].isna().sum()
    porcentaje_nulos = (total_feature_nulos / df.shape[0]) * 100
    print(f'Total de valores nulos de {feature}: {total_feature_nulos} ({porcentaje_nulos:.2f}%)')

print("------------------------------------------------")
print("Total de valores nulos en toda la data (después de eliminar columnas):", df.isnull().sum().sum())
print("Total de datos en toda la data (después de eliminar columnas):", df.size)
print("Columnas eliminadas:", columns_to_drop)


In [0]:
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()


### Inputación de variables categoricas

---



Las variables categóricas, tales como **'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', y 'RainTomorrow'**, requieren una imputación de valores nulos que mantenga la coherencia con la naturaleza de los datos. Dado que estas variables son nominales y representan categorías discretas, la imputación por **moda** es la elección adecuada. La **moda** representa el valor más frecuente en un conjunto de datos, lo que garantiza que la imputación preserve la distribución original de las categorías.

La imputación por **moda** se justifica porque preserva la distribución original de las categorías, asegurando que los valores imputados reflejen la tendencia predominante en los datos observados.

In [0]:

# Imputación de valores nulos para las variables categóricas por moda
categorical_variables = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
for var in categorical_variables:
    df[var].fillna(df[var].mode()[0], inplace=True)

# Verificar la cantidad de valores nulos después de la imputación
print("Valores nulos después de la imputación:")
print(df[['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']].isnull().sum())


In [0]:
#Matriz despues de haber imputado los nulos de variables categoricas
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación sin nulos catégoricos', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

### Imputación de Variables Numericas
---

In [0]:
# Definir las columnas numéricas
numerical_cols = [
    'MinTemp', 'MaxTemp', 'Rainfall',
    'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
    'Humidity3pm', 'Pressure9am', 'Pressure3pm',
    'Temp9am', 'Temp3pm', 'RISK_MM'
]

# Visualizar distribuciones de variables
plt.figure(figsize=(20, 15))

for i, col in enumerate(numerical_cols, 1):
    plt.subplot(5, 4, i)
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(col)

plt.tight_layout()
plt.show()



Para realizar la imputacion de los valores nulos para las variables numericas, se realizo un analisis mediante histogramas para visualizar la distribución de cada variable numérica en el dataset, con el cual se permite tomar una desicion para imputar segun corresponda a la distibución de cada variable:

 **1. Variables con Distribución Aproximadamente Normal**

**Variables: `MinTemp`, `MaxTemp`, `Temp9am`, `Temp3pm`, `RISK_MM`, `Rainfall`**

Estas variables presentan distribuciones aproximadamente normales. En estos casos, la **media** es una buena opción para la imputación de valores faltantes, ya que es representativa del centro de la distribución y permite mantener la simetría de los datos.

**2. Variables con Distribución Sesgada a la Derecha**

**Variables:  `Evaporation`, `WindGustSpeed`, `WindSpeed9am`, `WindSpeed3pm`**

Estas variables presentan distribuciones sesgadas a la derecha, con muchos valores cercanos a cero. En estos casos, la **mediana** es preferible para la imputación, ya que es menos sensible a los valores extremos y proporciona una medida más robusta del centro de la distribución.

**3. Variables con Distribuciones Sesgadas y Difíciles de Categorizar**

**Variables: `Sunshine`, `Humidity9am`, `Humidity3pm`, `Pressure9am`, `Pressure3pm`, `Cloud9am`, `Cloud3pm`**

Para estas variables, aunque presentan distribuciones sesgadas, la imputación por **mediana** es generalmente más adecuada debido a su robustez ante valores atípicos. Dado que la **mediana** no se ve afectada por extremos, es una opción segura para mantener la integridad de los datos.



In [0]:
# Imputación de valores nulos para variables con distribución normal
normal_variables = ['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm', 'RISK_MM', 'Rainfall']
for var in normal_variables:
    df[var].fillna(df[var].mean(), inplace=True)

In [0]:
#Matriz despues de haber imputado por la media
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

In [0]:

# Imputación de valores nulos para variables con distribución sesgada a la derecha
skewed_variables = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm']
for var in skewed_variables:
    df[var].fillna(df[var].median(), inplace=True)

# Imputación de valores nulos para variables con distribuciones sesgadas y difíciles de categorizar
difficult_variables = ['Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']
for var in difficult_variables:
    df[var].fillna(df[var].median(), inplace=True)


# Verificar la cantidad de valores nulos después de la imputación
print("Valores nulos después de la imputación:")
print(df[['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm',
          'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
          'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']].isnull().sum())


In [0]:
#Matriz despues de haber imputado por la mediana
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

###Imputación de Outlayers

In [0]:
def treat_outliers_with_mean(df):
    variables_interes = ['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm']
    for variable in variables_interes:
        if df[variable].dtype != 'object':  # Solo trabajamos con columnas numéricas
            Q1 = df[variable].quantile(0.25)
            Q3 = df[variable].quantile(0.75)
            IQR = Q3 - Q1

            # Determinar los límites inferior y superior
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Identificar outliers
            outliers_lower = df[df[variable] < lower_bound]
            outliers_upper = df[df[variable] > upper_bound]

            # Calcular la media de la columna sin considerar los outliers
            mean_value = df[(df[variable] >= lower_bound) & (df[variable] <= upper_bound)][variable].mean()

            # Reemplazar outliers por la media
            df.loc[outliers_lower.index, variable] = mean_value
            df.loc[outliers_upper.index, variable] = mean_value



treat_outliers_with_mean(df)



In [0]:
def treat_outliers_with_median(df):
    variables_interes = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']
    for variable in variables_interes:
        if df[variable].dtype != 'object':  # Solo trabajamos con columnas numéricas
            Q1 = df[variable].quantile(0.25)
            Q3 = df[variable].quantile(0.75)
            IQR = Q3 - Q1

            # Determinar los límites inferior y superior
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Identificar outliers
            outliers_lower = df[df[variable] < lower_bound]
            outliers_upper = df[df[variable] > upper_bound]

            # Calcular la media de la columna sin considerar los outliers
            median_value = df[(df[variable] >= lower_bound) & (df[variable] <= upper_bound)][variable].median()

            # Reemplazar outliers por la media
            df.loc[outliers_lower.index, variable] = median_value
            df.loc[outliers_upper.index, variable] = median_value



treat_outliers_with_median(df)


In [0]:
def treat_outliers(df):
    variables_interes = [ 'Rainfall','RISK_MM']
    for variable in variables_interes:
        if df[variable].dtype != 'object':  # Solo trabajamos con columnas numéricas
            Q1 = df[variable].quantile(0.25)
            Q3 = df[variable].quantile(0.75)
            IQR = Q3 - Q1

            # Determinar los límites inferior y superior
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Identificar outliers
            outliers_lower = df[df[variable] < lower_bound]
            outliers_upper = df[df[variable] > upper_bound]

            # Reemplazar outliers por el valor correspondiente al límite inferior o superior
            df.loc[outliers_lower.index, variable] = lower_bound
            df.loc[outliers_upper.index, variable] = upper_bound

#Llamar a la función para tratar outliers
treat_outliers(df)

In [0]:
def print_outliers(df):
    print("\nTOTAL DE OUTLIERS")
    for variable in df:
        if df[variable].dtype != 'object':  # Solo trabajamos con columnas numéricas
            Q1 = df[variable].quantile(0.25)
            Q3 = df[variable].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = df[(df[variable] < lower_bound) | (df[variable] > upper_bound)]
            total_outliers = len(outliers)
            print(f"Total de outliers de {variable}: {total_outliers}")

# Llamamos a la función para imprimir los outliers
print_outliers(df)

In [0]:
#Matriz despues de haber imputado los Outlaiers
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

###**<h1>Ingeniería de características</h1>**

**DATE** <BR>
Es importante desglosar la fecha en diferentes columnas (día, mes, año) porque nos permite entender mejor cómo cambian las cosas con el tiempo.

Al dividir la fecha en partes más pequeñas y poner cada parte en su propia columna, es más fácil ver estos patrones. Por ejemplo, podrías mirar cuánta lluvia cae en promedio en cada mes o cómo han cambiado las temperaturas cada año.

In [0]:
# Convertir la columna "Date" a formato de fecha
df['Date'] = pd.to_datetime(df['Date'])

# Crear nuevas columnas para día, mes y año
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Imprimir las primeras filas para verificar
print(df[['Date', 'Day', 'Month', 'Year']].head())

In [0]:
df['Date'] = pd.to_datetime(df['Date'])

# Función para obtener la estación del año en el
#hemisferio sur (Australia)
def obtener_estacion_australia(fecha):
    mes = fecha.month
    if mes in (12, 1, 2):
        return 'Verano'
    elif mes in (3, 4, 5):
        return 'Otoño'
    elif mes in (6, 7, 8):
        return 'Invierno'
    else:
        return 'Primavera'

# Aplicamos la función a la columna 'Date' y guardamos el
#resultado en una nueva columna llamada 'Season'
df['Season'] = df['Date'].apply(obtener_estacion_australia)

# Muestra el DataFrame resultante
df["Season"].head()

Despues de hacer esta ingeniería de caracteristicas, borraremos la variable Date, ya que no la necesitaremos al tener las variables Day, Month, Year

In [0]:
# Eliminar la variable 'variable_a_eliminar' del DataFrame df
df.drop(columns=['Date'], inplace=True)


### **LaberEncoder()**


In [0]:
from sklearn.preprocessing import LabelEncoder

# Lista de variables categóricas en tu DataFrame
categorical_variables = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow', 'Season']

# Creamos el codificador LabelEncoder
encoder = LabelEncoder()

# Aplicamos el LabelEncoder a cada variable categórica en el DataFrame
for var in categorical_variables:
    df[var] = encoder.fit_transform(df[var])


print(df[categorical_variables])

In [0]:
#Matriz despues de haber realizado LabelEncoder()
#Seleccionar las variables no categóricas
variables_no_categoricas = ['RISK_MM', 'Humidity3pm',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm','Rainfall']

# Filtrar el DataFrame para incluir solo las variables no categóricas
df_no_categoricas = df[variables_no_categoricas]

# Calcular la matriz de correlación para las variables no categóricas
corr_matrix_no_categoricas = df_no_categoricas.corr()

# Crear una figura de tamaño adecuado
plt.figure(figsize=(12, 10))

# Generar el mapa de calor de la matriz de correlación para las variables no categóricas
sns.heatmap(corr_matrix_no_categoricas, annot=True, cmap='coolwarm')

# Añadir título y etiquetas
plt.title('Matriz de Correlación', fontsize=16)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)

# Mostrar el gráfico
plt.show()

## 4.- Modelos

In [0]:
#Importar librerias
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Modelos de RISK_MM

In [0]:
X = df.drop('RISK_MM', axis=1)
y = df['RISK_MM']

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###Regresion Lineal

In [0]:
# Crear y entrenar el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_predictions = linear_model.predict(X_test)

# Calcular métricas de regresión lineal
linear_rmse = np.sqrt(mean_squared_error(y_test, linear_predictions))
linear_mse = mean_squared_error(y_test, linear_predictions)
linear_mae = mean_absolute_error(y_test, linear_predictions)
linear_r2 = r2_score(y_test, linear_predictions)

# Imprimir métricas de regresión lineal
print(f'Regresión Lineal Metrics:')
print(f'RMSE: {linear_rmse}')
print(f'MSE: {linear_mse}')
print(f'MAE: {linear_mae}')
print(f'R^2 Score: {linear_r2}')


###KNN

In [0]:
# Crear y entrenar el modelo de KNN para regresión
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)
knn_predictions = knn_model.predict(X_test)

# Calcular métricas de KNN
knn_rmse = np.sqrt(mean_squared_error(y_test, knn_predictions))
knn_mse = mean_squared_error(y_test, knn_predictions)
knn_mae = mean_absolute_error(y_test, knn_predictions)
knn_r2 = r2_score(y_test, knn_predictions)

# Imprimir métricas de KNN
print(f'\nKNN Metrics:')
print(f'RMSE: {knn_rmse}')
print(f'MSE: {knn_mse}')
print(f'MAE: {knn_mae}')
print(f'R^2 Score: {knn_r2}')


###RANDOM FOREST

In [0]:

# Crear y entrenar el modelo de bosques aleatorios para regresión
forest_model = RandomForestRegressor()
forest_model.fit(X_train, y_train)
forest_predictions = forest_model.predict(X_test)

# Calcular métricas de bosques aleatorios
forest_rmse = np.sqrt(mean_squared_error(y_test, forest_predictions))
forest_mse = mean_squared_error(y_test, forest_predictions)
forest_mae = mean_absolute_error(y_test, forest_predictions)
forest_r2 = r2_score(y_test, forest_predictions)

# Imprimir métricas de bosques aleatorios
print(f'\nBosques Aleatorios Metrics:')
print(f'RMSE: {forest_rmse}')
print(f'MSE: {forest_mse}')
print(f'MAE: {forest_mae}')
print(f'R^2 Score: {forest_r2}')


####<h2>Métricas de Regresión</h2>
<table>
    <tr>
        <th>Métrica</th>
        <th>Descripción</th>
        <th>Rango</th>
    </tr>
    <tr>
        <td>RMSE (Error Cuadrático Medio)</td>
        <td>Mide cuánto se equivoca nuestro modelo en sus predicciones, considerando tanto las predicciones demasiado altas como las demasiado bajas. Cuanto más bajo sea, mejor.</td>
        <td>
            <ul>
                <li>Menor que 1: Excelente</li>
                <li>Entre 1 y 5: Muy bueno</li>
                <li>Entre 5 y 10: Bueno</li>
                <li>Mayor que 10: Aceptable</li>
                <li>Mayor que 20: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>MSE (Error Cuadrático Medio)</td>
        <td>Es similar al RMSE, pero no tiene la raíz cuadrada. Mide el promedio de los errores al cuadrado. Cuanto más bajo, mejor.</td>
        <td>
            <ul>
                <li>Menor que 1: Excelente</li>
                <li>Entre 1 y 25: Muy bueno</li>
                <li>Entre 25 y 100: Bueno</li>
                <li>Mayor que 100: Aceptable</li>
                <li>Mayor que 400: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>MAE (Error Absoluto Medio)</td>
        <td>Mide cuánto se equivoca nuestro modelo en promedio, sin considerar la dirección de los errores. Cuanto más bajo sea, mejor.</td>
        <td>
            <ul>
                <li>Menor que 1: Excelente</li>
                <li>Entre 1 y 5: Muy bueno</li>
                <li>Entre 5 y 10: Bueno</li>
                <li>Mayor que 10: Aceptable</li>
                <li>Mayor que 20: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>R^2 Score (Coeficiente de Determinación)</td>
        <td>Indica cuánta variación en los datos puede explicar nuestro modelo. Cuanto más cerca de 1, mejor.</td>
        <td>
            <ul>
                <li>Mayor que 0.9: Excelente</li>
                <li>Entre 0.8 y 0.9: Muy bueno</li>
                <li>Entre 0.7 y 0.8: Bueno</li>
                <li>Entre 0.6 y 0.7: Aceptable</li>
                <li>Menor que 0.6: Cuestionable</li>
            </ul>
        </td>
    </tr>
</table>

## Modelos de Rain_tomorrow
---

In [0]:
#Importar librerias
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
X = df.drop('RainTomorrow', axis=1)
y = df['RainTomorrow']

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Regresion Logistica

In [0]:
# Crear y entrenar el modelo de regresión logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_predictions = logistic_model.predict(X_test)

# Calcular métricas de regresión logística
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_precision = precision_score(y_test, logistic_predictions)
logistic_recall = recall_score(y_test, logistic_predictions)
logistic_f1 = f1_score(y_test, logistic_predictions)

# Imprimir métricas de regresión logística
print(f'Regresión Logística Metrics:')
print(f'Accuracy: {logistic_accuracy}')
print(f'Precision: {logistic_precision}')
print(f'Recall: {logistic_recall}')
print(f'F1 Score: {logistic_f1}')



###KNN

In [0]:
# Crear y entrenar el modelo de KNN para clasificación
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_test)

# Calcular métricas de KNN
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions)
knn_recall = recall_score(y_test, knn_predictions)
knn_f1 = f1_score(y_test, knn_predictions)

# Imprimir métricas de KNN
print(f'\nKNN Metrics:')
print(f'Accuracy: {knn_accuracy}')
print(f'Precision: {knn_precision}')
print(f'Recall: {knn_recall}')
print(f'F1 Score: {knn_f1}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, knn_predictions)}')


### Super Vector Machine

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Crear y entrenar el modelo SVM para clasificación
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

# Calcular métricas de SVM
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_f1 = f1_score(y_test, svm_predictions)

# Imprimir métricas de SVM
print(f'\nSVM Metrics:')
print(f'Accuracy: {svm_accuracy}')
print(f'Precision: {svm_precision}')
print(f'Recall: {svm_recall}')
print(f'F1 Score: {svm_f1}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, svm_predictions)}')


<h2>Métricas de Clasificación</h2>
<table>
    <tr>
        <th>Métrica</th>
        <th>Descripción</th>
        <th>Rango</th>
    </tr>
    <tr>
        <td>Accuracy (Exactitud)</td>
        <td>Indica la proporción de predicciones correctas respecto al total. Cuanto más alto, mejor.</td>
        <td>
            <ul>
                <li>Mayor que 0.9: Excelente</li>
                <li>Entre 0.8 y 0.9: Muy bueno</li>
                <li>Entre 0.7 y 0.8: Bueno</li>
                <li>Entre 0.6 y 0.7: Aceptable</li>
                <li>Menor que 0.6: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>Precision (Precisión)</td>
        <td>Mide la proporción de verdaderos positivos sobre el total de predicciones positivas. Cuanto más alto, mejor. Es útil en contextos donde los falsos positivos son costosos.</td>
        <td>
            <ul>
                <li>Mayor que 0.9: Excelente</li>
                <li>Entre 0.8 y 0.9: Muy bueno</li>
                <li>Entre 0.7 y 0.8: Bueno</li>
                <li>Entre 0.6 y 0.7: Aceptable</li>
                <li>Menor que 0.6: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>Recall (Sensibilidad o Tasa de Verdaderos Positivos)</td>
        <td>Mide la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos. Cuanto más alto, mejor. Es útil en contextos donde los falsos negativos son costosos.</td>
        <td>
            <ul>
                <li>Mayor que 0.9: Excelente</li>
                <li>Entre 0.8 y 0.9: Muy bueno</li>
                <li>Entre 0.7 y 0.8: Bueno</li>
                <li>Entre 0.6 y 0.7: Aceptable</li>
                <li>Menor que 0.6: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>F1 Score</td>
        <td>Es la media armónica de precisión y recall. Proporciona una medida balanceada para modelos con clases desbalanceadas. Cuanto más alto, mejor.</td>
        <td>
            <ul>
                <li>Mayor que 0.9: Excelente</li>
                <li>Entre 0.8 y 0.9: Muy bueno</li>
                <li>Entre 0.7 y 0.8: Bueno</li>
                <li>Entre 0.6 y 0.7: Aceptable</li>
                <li>Menor que 0.6: Cuestionable</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td>Confusion Matrix (Matriz de Confusión)</td>
        <td>Proporciona una tabla con los valores de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos. No tiene un rango específico, pero debe ser interpretada para entender la distribución de errores.</td>
        <td>
            <ul>
                <li>N/A: Debe ser interpretada en el contexto del problema específico.</li>
            </ul>
        </td>
    </tr>
</table>
